<a href="https://colab.research.google.com/github/aknsntn/math_ml/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import re
import numpy as np

import nltk
from nltk import ngrams
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Подготовка данных

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/sms_spam.csv')

In [ ]:
df

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
...,...,...
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."


In [ ]:
df.isnull().sum()

type    0
text    0
dtype: int64

In [ ]:
stopwords = stopwords.words('english')
stemmer = SnowballStemmer(language = 'english')

In [ ]:
def clear(row):
  row = row.lower()
  row = re.sub('[^a-zA-z]' , ' ' , row)
  token = word_tokenize(row)
  titles = [stemmer.stem(word) for word in token if word not in stopwords]

  prep_text = ' '.join(titles)

  return prep_text

In [ ]:
X = df['text'].apply(lambda x : clear(x))
y = df['type']

In [ ]:
X

0                                    hope good week check
1                                       k give back thank
2                                            also cbe pay
3       complimentari star ibiza holiday cash need urg...
4       okmail dear dave final notic collect tenerif h...
                              ...                        
5554    great role model give much realli wish day mir...
5555    awesom rememb last time got somebodi high firs...
5556    prize go anoth custom c www c biz p min polo l...
5557    sms ac jsco energi high u may know channel day...
5558                                 shall call dear food
Name: text, Length: 5559, dtype: object

In [ ]:
y

0        ham
1        ham
2        ham
3       spam
4       spam
        ... 
5554     ham
5555     ham
5556    spam
5557    spam
5558     ham
Name: type, Length: 5559, dtype: object

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = MultinomialNB()

#### Мешок n-грамм

In [ ]:
pipeline1 = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("clf", clf),
    ]
)

parameters = {
    "vect__max_df": (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 2), (1, 3)),
    }

grid_search = GridSearchCV(pipeline1, parameters, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__max_features': (None, 5000, 10000, 50000),
                         'vect__ngram_range': ((1, 2), (1, 3))},
             verbose=1)

In [ ]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters1 = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters1[param_name]))

Best score: 0.985
Best parameters set:
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 2)


In [ ]:
ngram = Pipeline(
    [
        ("vect", CountVectorizer(max_df = 0.5, max_features = None, ngram_range = (1, 2))),
        ("clf", clf),
    ]
)

ngram.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(max_df=0.5, ngram_range=(1, 2))),
                ('clf', MultinomialNB())])

In [ ]:
n_pred = ngram.predict(X_test)

#### Tf-Idf

In [ ]:
pipeline2 = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", clf),
    ]
)

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__min_df': (0.0, 0.5, 0.75, 1.0),
    "vect__ngram_range": ((1, 2), (1, 3)),
    }

grid_search = GridSearchCV(pipeline2, parameters, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)


In [ ]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters2 = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters2[param_name]))

Best score: 0.963
Best parameters set:
	vect__max_df: 0.5
	vect__max_features: 5000
	vect__min_df: 0.0
	vect__ngram_range: (1, 2)


In [ ]:
tfidf = pipeline2 = Pipeline(
    [
        ("vect", TfidfVectorizer(max_df = 0.5, max_features = 5000, min_df = 0.0, ngram_range = (1, 2))),
        ("clf", clf),
    ]
)

tfidf.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.5, max_features=5000, min_df=0.0,
                                 ngram_range=(1, 2))),
                ('clf', MultinomialNB())])

In [ ]:
tf_pred = tfidf.predict(X_test)

#### Символьные n-граммы

In [ ]:
pipeline3 = Pipeline(
    [
        ("vect", CountVectorizer(analyzer = 'char')),
        ("clf", clf),
    ]
)

parameters = {
    "vect__max_df": (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 5), (3, 6), (2, 8), (4, 10)),
    }

grid_search = GridSearchCV(pipeline3, parameters, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters3 = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters3[param_name]))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best score: 0.972
Best parameters set:
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (2, 8)


In [ ]:
chargrams = Pipeline(
    [
        ("vect", CountVectorizer(analyzer = 'char', max_df = 0.5, max_features = None, ngram_range = (2, 8))),
        ("clf", clf),
    ]
)

chargrams.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(analyzer='char', max_df=0.5,
                                 ngram_range=(2, 8))),
                ('clf', MultinomialNB())])

In [ ]:
ch_pred = chargrams.predict(X_test)

#### Сравнение векторайзеров

In [ ]:
result = pd.DataFrame(index = [['Мешок n-грамм', 'TF-IDF', 'Символьные n-граммы']]) # precision, recall, f1-score и accuracy

In [ ]:
result['precision'] = [(precision_score(y_test, n_pred, average = 'macro')),
                       (precision_score(y_test, tf_pred, average = 'macro')), 
                       (precision_score(y_test, ch_pred, average = 'macro'))]

result['recall'] = [(recall_score(y_test, n_pred, average = 'macro')),
                    (recall_score(y_test, tf_pred, average = 'macro')), 
                    (recall_score(y_test, ch_pred, average = 'macro'))]

result['f1-score'] = [(f1_score(y_test, n_pred, average = 'macro')),
                      (f1_score(y_test, tf_pred, average = 'macro')), 
                      (f1_score(y_test, ch_pred, average = 'macro'))]

result['accuracy'] = [(accuracy_score(y_test, n_pred)),
                      (accuracy_score(y_test, tf_pred)), 
                      (accuracy_score(y_test, ch_pred))]

In [ ]:
result

,precision,recall,f1-score,accuracy
Мешок n-грамм,0.983210,0.964934,0.973820,0.988010
TF-IDF,0.983132,0.906320,0.940087,0.974221
Символьные n-граммы,0.953763,0.953763,0.953763,0.978417


Исходя из значений таблицы, можно сделать вывод, что мешок n-грамм показал себя лучше всего.

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

потому что a в abcx является окончанием abca

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
re.findall(r'\b\S.', 'иногда всем бывает очень тяжело')

['ин', 'вс', 'бы', 'оч', 'тя']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
text = 'Связаны. Связаны. Клеток, связанных внутри. Клеток, связанных внутри. Повторите три раза. Клеток, связанных внутри. Клеток, связанных внутри. Клеток, связанных внутри.'
re.split(r'[.]\s', text, maxsplit=2)

['Связаны',
 'Связаны',
 'Клеток, связанных внутри. Клеток, связанных внутри. Повторите три раза. Клеток, связанных внутри. Клеток, связанных внутри. Клеток, связанных внутри.']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
text2 = '112 один один два 45 ^'
re.sub('\d+', 'DIG', text2)

'DIG один один два DIG ^'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
url = 'https://psi.mchs.gov.ru/psihologicheskaya-sluzhba/voprosy-k-psihologu интересный веб-ресурс, к которому есть смысл обращаться во время учебы'
re.sub(r'http\S+.', '', url)

'интересный веб-ресурс, к которому есть смысл обращаться во время учебы'

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog2 = re.compile('\S\w[А-Яа-яё\-]+')
prog2.findall("Слова? Да, больше, ещё больше слов! Что-то ещё. к, 6, м, пу, много!")

['Слова', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё', 'много']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
domens = re.compile(r'@[\w-]+\.[\w.-]+')
domens.findall('abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']